In [1]:
import pandas as pd
import numpy as np

In [2]:
def readArff(filename):
    with open ('./NominalData/'+filename+'.arff', 'r') as f:
        # split lines, remove ones with comments
        lines = [line.lower() for line in f.read().split('\n') if not line.startswith('%')]
        
    # remove empty lines
    lines = [line for line in lines if line != '']
    
    columns = []
    data = []
    for index, line in enumerate(lines):
        if line.startswith('@attribute'):
            columns.append(line)
            
        if line.startswith('@data'):
            # get the rest of the lines excluding the one that says @data
            data = lines[index+1:]
            break
            
    # clean column names -- '@attribute colname  \t\t\t{a, b, ...}'
    cleaned_columns = [c[11:c.index('{')].strip() for c in columns]
    
    # clean and split data
    cleaned_data = [d.replace(', ', ',').split(',') for d in data]
    
    # create dataframe
    return pd.DataFrame(cleaned_data, columns = cleaned_columns)

In [3]:
def handle_attributes(data):
    """
    Method to turn nominal values into binary valued attributes using the get_dummies function
    
    Returns numpy 2d array
    """
    xs = pd.get_dummies(data.iloc[:,:-1])
    y, unique = pd.factorize(data.iloc[:,-1])
    
    # merge back together
    final = pd.concat([xs, pd.DataFrame(y, columns=[data.columns[-1]])], axis=1)
    return final.values

In [4]:
def accuracy(list1, list2):
    """
    Computes the element-wise similarity between lists list1 and list2.
    INPUTS:
    - list1 [list]: list of arbitrary values
    - list2 [list]: list of arbitrary values
    RETURNS:
    - n_correct [int]: total num. of elementwise matches
    - n_total [int]: total num. elements compared
    """
    assert len(list1) == len(list2)
    n_total = len(list1)
    n_correct = sum([list1[i] == list2[i] for i in range(n_total)])
    n_incorrect = n_total-n_correct
    print(f"Total Correct: {n_correct}/{n_total} = {(n_correct/n_total)*100}%")
    print(f"Total Incorrect: {n_incorrect}/{n_total} = {(n_incorrect/n_total)*100}%")

In [5]:
def sigmoid(x, deriv=False):
    if(deriv):
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [6]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    """
    Data Structure:
    1 hidden layer, 1 output layer
    network = [hidden_layer, output_layer]
    hidden_layer = [{weights: [1,2,3]},
                    {weights: [1,2,3]},
                    ...]
    output_layer = [{weights: [1,2,3]},
                    {weights: [1,2,3]},
                    ...]
    
    *Note:* add 1 to n_inputs for w0 aka the bias which will be the *last* element in the array
    """
    
    hidden_layer = [{'weights': np.random.uniform(low=-0.05, high=0.05, size = (n_inputs+1, ))} 
                   for i in range(n_hidden)]
    
    output_layer = [{'weights': np.random.uniform(low=-0.05, high=0.05, size = (n_hidden+1, ))} 
                    for i in range(n_outputs)]
    return hidden_layer, output_layer

In [7]:
def calculate_linear_unit(weights, instances):
    """Compute the output unit 
    Calculated as a linear combination of the weights and instances
    Bias is assumed to be the last element in weights
    output_u = bias + sum(weight_i * input_i)
    """
    output = weights[-1]
    for i in range(len(weights)-1):
        output += weights[i] * instances[i]
    return output

In [8]:
def propagate_forward(hidden_layer, output_layer, row):
    """
    Propagates inputs forward through hidden layer then through output layer
    Returns final output and saves output in dictionary at each neuron.
    """
    instances = row
    new_instances = []
    
    for neuron in hidden_layer:
        activation = calculate_linear_unit(neuron['weights'], instances)
        neuron['output'] = sigmoid(activation)
        new_instances.append(neuron['output'])
    
    final = []
    for neuron in output_layer:
        activation = calculate_linear_unit(neuron['weights'], new_instances)
        neuron['output'] = sigmoid(activation)
        final.append(neuron['output'])
    
    return final

In [9]:
def calculate_errors(layer, errors):
    """
    Helper function for propagate_backwards
    
    Calculates error terms and stores it in the layer
    """
    for i in range(len(layer)):
        neuron = layer[i]
        neuron['delta'] = errors[i] * sigmoid(neuron['output'], deriv=True)
    return layer

In [10]:
def propagate_backward(hidden_layer, output_layer, expected):
    """
    Propagates errors back through the network
    Output layer
    """
    errors1 = []
    for i in range(len(output_layer)):
        neuron = output_layer[i]
        errors1.append(expected[i] - neuron['output'])
        
    output_layer = calculate_errors(output_layer, errors1)
    
    errors2 = []
    for i in range(len(hidden_layer)):
        error = 0.0
        for neuron in output_layer:
            error += (neuron['weights'][i] * neuron['delta'])
        errors2.append(error)
        
    hidden_layer = calculate_errors(hidden_layer, errors2)

In [11]:
def update_layer_weights(layer, instances, l_rate):
    """
    Helper function for update_weights
    """
    for neuron in layer:
        for i in range(len(instances)):
            neuron['weights'][i] += l_rate * neuron['delta'] * instances[i]
        # update bias
        neuron['weights'][-1] += l_rate * neuron['delta']

In [12]:
def update_weights(hidden_layer, output_layer, row, l_rate):
    """
    Updates network weights with error based on equation:
    w <-- w + n*delta*x
    """
    
    instances1 = row[:-1]
    update_layer_weights(hidden_layer, instances1, l_rate)
    
    instances2 = [neuron['output'] for neuron in hidden_layer]
    update_layer_weights(output_layer, instances2, l_rate)

In [13]:
def train_network(hidden_layer, output_layer, data, l_rate, n_epoch, n_outputs):
    """
    Repeat backprop algorithm as outlined in mitchell for n_epoch iterations
    """
    for epoch in range(n_epoch):
        for row in data:
            outputs = propagate_forward(hidden_layer, output_layer, row)
            expected = [0]*n_outputs
            expected[row[-1]] = 1
            propagate_backward(hidden_layer, output_layer, expected)
            update_weights(hidden_layer, output_layer, row, l_rate)

In [14]:
def predict(hidden_layer, output_layer, row):
    outputs = propagate_forward(hidden_layer, output_layer, row)
    return outputs.index(max(outputs))

In [15]:
def full_dataset_test(dataset):
    """
    Train on all rows. Test on all rows. 
    
    Combine results for accuracy.
    """
    n_inputs = len(dataset[0]) - 1
    n_outputs = len(set([row[-1] for row in dataset]))
    hidden_layer, output_layer = initialize_network(n_inputs, 2, n_outputs)
    train_network(hidden_layer, output_layer, dataset, 0.05, 200, n_outputs)
    
    predictions = []
    actual = []
    for row in dataset:
        predictions.append(predict(hidden_layer, output_layer, row))
        actual.append(row[-1])
    return accuracy(predictions, actual)

In [16]:
def hold_one_out(dataset):
    """
    Hold-one-out cross validation.
    
    Returns average results.
    """
    n_inputs = len(dataset[0]) - 1
    n_outputs = len(set([row[-1] for row in dataset]))
    predictions = []
    actual = []
    
    for i, instance in enumerate(dataset):
        subset = np.delete(dataset, i, axis=0)
        hidden_layer, output_layer = initialize_network(n_inputs, 2, n_outputs)
        train_network(hidden_layer, output_layer, subset, 0.05, 200, n_outputs)
        predictions.append(predict(hidden_layer, output_layer, instance))
        actual.append(instance[-1])
    return accuracy(predictions, actual)

In [17]:
def ten_fold_cross_validation(dataset):
    """
    10-fold cross validation. Splits data into 10 segments. Trains on 9 and tests on 1.
    
    Returns an average of all results for each 10 tests.
    """
    n_inputs = len(dataset[0]) - 1
    n_outputs = len(set([row[-1] for row in dataset]))
    predictions = []
    actual = []
    
    step = len(dataset)//10
    indices = [i for i in range(0, len(dataset), step)]
    prev = 0
    count = 1
    
    for i in indices[1:]:
        print(f"Subset: {count}")
        subset = np.delete(dataset, slice(prev, i), axis=0)
        hidden_layer, output_layer = initialize_network(n_inputs, 2, n_outputs)
        train_network(hidden_layer, output_layer, subset, 0.05, 200, n_outputs)
        
        for instance in dataset[prev:i, :]:
            predictions.append(predict(hidden_layer, output_layer, instance))
            actual.append(instance[-1])
        prev = i
        count += 1
    return accuracy(predictions, actual)

In [26]:
weather = handle_attributes(readArff('weather.nominal'))
full_dataset_test(weather)

Total Correct: 9/14 = 64.28571428571429%
Total Incorrect: 5/14 = 35.714285714285715%


In [27]:
hold_one_out(weather)

Total Correct: 9/14 = 64.28571428571429%
Total Incorrect: 5/14 = 35.714285714285715%


In [28]:
lenses = handle_attributes(readArff('contact-lenses'))

In [29]:
full_dataset_test(lenses)

Total Correct: 18/24 = 75.0%
Total Incorrect: 6/24 = 25.0%


In [30]:
hold_one_out(lenses)

Total Correct: 13/24 = 54.166666666666664%
Total Incorrect: 11/24 = 45.83333333333333%


In [31]:
titanic = handle_attributes(readArff('titanic'))

In [32]:
full_dataset_test(titanic)

Total Correct: 597/712 = 83.84831460674157%
Total Incorrect: 115/712 = 16.15168539325843%


In [33]:
ten_fold_cross_validation(titanic)

Subset: 1
Subset: 2
Subset: 3
Subset: 4
Subset: 5
Subset: 6
Subset: 7
Subset: 8
Subset: 9
Subset: 10
Total Correct: 557/710 = 78.45070422535211%
Total Incorrect: 153/710 = 21.549295774647888%
